# 第7回：最終プロジェクトと音楽発表セッション

## 本日の目標
- これまでの知識を統合して、オリジナル楽曲を制作する
- 創作プロセスを体験し、技術と創造性の融合を学ぶ
- 作品を発表し、お互いの作品を聞き合う

---

## セットアップ

In [2]:
# 環境検出とセットアップ
import sys
import os

# Google Colab環境の検出
try:
    import google.colab
    IN_COLAB = True
    print("🔧 Google Colab環境で実行中...")
except ImportError:
    IN_COLAB = False
    print("🏠 ローカル環境で実行中")

# ライブラリのセットアップ
if IN_COLAB:
    print("🔧 Google Colab環境を設定中...")
    
    # 必要なパッケージをインストール
    !pip install numpy scipy matplotlib ipython japanize-matplotlib librosa soundfile
    
    # GitHubからライブラリをクローン
    !git clone https://github.com/ggszk/simple-audio-programming.git
    
    # パスを追加
    sys.path.append('/content/simple-audio-programming')
    
    print("✅ セットアップ完了！")
    print("📝 このノートブックを自分用にコピーするには:")
    print("   ファイル → ドライブにコピーを保存")
    
else:
    # ローカル環境では、Poetry環境であることを前提とする
    print("📋 ローカル環境での前提条件:")
    print("   1. Poetryがインストールされていること")
    print("   2. プロジェクトディレクトリで 'poetry install' が実行済みであること")
    print("   3. 'poetry shell' で仮想環境がアクティブであること")
    print("   4. または 'poetry run jupyter lab' で起動していること")
    
    # 現在のPythonパスを表示
    print(f"\n🔍 デバッグ情報:")
    print(f"Python実行ファイル: {sys.executable}")
    print(f"作業ディレクトリ: {os.getcwd()}")
    
    try:
        import audio_lib
        print("✅ audio_lib ライブラリが利用可能です")
        print(f"audio_lib パス: {audio_lib.__file__}")
    except ImportError as e:
        print("❌ audio_lib ライブラリが見つかりません")
        print(f"エラー: {e}")
        print("📖 セットアップ手順:")
        print("   1. ターミナルでプロジェクトディレクトリに移動")
        print("   2. poetry install")
        print("   3. poetry shell")
        print("   4. jupyter lab")
        raise ImportError("Poetry環境が正しくセットアップされていません")

# インポート文
print("\n📦 必要なクラスをimport中...")
try:
    from audio_lib import (
        AudioConfig, SineWave, SquareWave, SawtoothWave, TriangleWave, ADSREnvelope,
        note_to_frequency, note_name_to_number, save_audio,
        Sequencer, Note, Track,  # シーケンサー関連クラス
        BasicPiano, BasicOrgan, BasicGuitar, BasicDrum,  # 楽器クラス
        LowPassFilter, HighPassFilter,  # フィルタークラス
        NoiseGenerator  # ノイズジェネレーター
    )
    print("✅ 全てのクラスのimportに成功しました")
    
    # インポートしたクラスを確認
    print(f"AudioConfig クラス: {AudioConfig}")
    print(f"SineWave クラス: {SineWave}")
    
except ImportError as e:
    print(f"❌ import エラー: {e}")
    # より詳細なエラー情報
    import traceback
    traceback.print_exc()

# 追加エフェクト（利用可能な場合）
try:
    from audio_lib.effects.audio_effects import Reverb, Distortion, Delay, Chorus
    print("✅ エフェクトクラスが利用可能です")
except ImportError:
    print("⚠️ 一部のエフェクトは利用できません")

# 追加インポート
from audio_lib.synthesis.envelopes import apply_envelope
from IPython.display import Audio, display
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import librosa
import soundfile as sf
import time

# 日本語フォント設定（Colab用）
if IN_COLAB:
    import japanize_matplotlib
    print("✅ 日本語フォントを設定しました")

import warnings
warnings.filterwarnings('ignore')

# AudioConfigとオシレーターの初期化
config = AudioConfig()
sine_osc = SineWave(config)
square_osc = SquareWave(config)
saw_osc = SawtoothWave(config)
triangle_osc = TriangleWave(config)

print(f"\n🎼 サンプリング周波数: {config.sample_rate}Hz")
print("🚀 最終プロジェクトのセットアップ完了！楽曲制作を始めましょう！")

🏠 ローカル環境で実行中
📋 ローカル環境での前提条件:
   1. Poetryがインストールされていること
   2. プロジェクトディレクトリで 'poetry install' が実行済みであること
   3. 'poetry shell' で仮想環境がアクティブであること
   4. または 'poetry run jupyter lab' で起動していること

🔍 デバッグ情報:
Python実行ファイル: /Users/gsuzuki/Library/Caches/pypoetry/virtualenvs/simple-audio-programming-zUlWXoPV-py3.13/bin/python
作業ディレクトリ: /Users/gsuzuki/projects/teaching/simple-audio-programming/colab_lessons
✅ audio_lib ライブラリが利用可能です
audio_lib パス: /Users/gsuzuki/projects/teaching/simple-audio-programming/audio_lib/__init__.py

📦 必要なクラスをimport中...
✅ 全てのクラスのimportに成功しました
AudioConfig クラス: <class 'audio_lib.core.audio_config.AudioConfig'>
SineWave クラス: <class 'audio_lib.synthesis.oscillators.SineWave'>
✅ エフェクトクラスが利用可能です

🎼 サンプリング周波数: 44100Hz
🚀 最終プロジェクトのセットアップ完了！楽曲制作を始めましょう！

🎼 サンプリング周波数: 44100Hz
🚀 最終プロジェクトのセットアップ完了！楽曲制作を始めましょう！


In [3]:
# 統合された音響ライブラリのヘルパー関数
class ProjectHelpers:
    """最終プロジェクト用のヘルパー関数集"""
    
    def __init__(self, config):
        self.config = config
        self.sample_rate = config.sample_rate
    
    def create_chord(self, base_freq, chord_type="major", duration=1.0, amplitude=0.3):
        """和音を生成"""
        if chord_type == "major":
            frequencies = [base_freq, base_freq * 5/4, base_freq * 3/2]  # 長三和音
        elif chord_type == "minor":
            frequencies = [base_freq, base_freq * 6/5, base_freq * 3/2]  # 短三和音
        else:
            frequencies = [base_freq]
        
        # 各音を生成して合成
        chord_audio = np.zeros(int(self.sample_rate * duration))
        for freq in frequencies:
            tone = sine_osc.generate(freq, duration, amplitude / len(frequencies))
            chord_audio += tone
        
        return chord_audio
    
    def create_melody_sequence(self, notes, durations, instrument=None):
        """メロディーシーケンスを生成"""
        if instrument is None:
            instrument = BasicPiano(self.config)
        
        audio_sequence = []
        for note, duration in zip(notes, durations):
            if isinstance(note, str):
                # 音階名の場合は周波数に変換
                freq = note_to_frequency(note_name_to_number(note))
            else:
                freq = note
            
            # 楽器で音を生成
            note_audio = instrument.play_note(freq, duration)
            audio_sequence.append(note_audio)
        
        # 全ての音を連結
        return np.concatenate(audio_sequence)
    
    def create_drum_pattern(self, pattern, bpm=120, bars=1):
        """ドラムパターンを生成"""
        beat_duration = 60.0 / bpm  # 1拍の長さ（秒）
        
        drum = BasicDrum(self.config)
        pattern_audio = []
        
        for bar in range(bars):
            for beat in pattern:
                if beat == 'K':  # キック
                    audio = drum.play_note(36, beat_duration * 0.8)  # C2（キック）
                elif beat == 'S':  # スネア
                    audio = drum.play_note(38, beat_duration * 0.6)  # D2（スネア）
                elif beat == 'H':  # ハイハット
                    audio = drum.play_note(42, beat_duration * 0.3)  # F#2（クローズドハイハット）
                else:  # 休符
                    audio = np.zeros(int(beat_duration * self.sample_rate))
                
                pattern_audio.append(audio)
        
        return np.concatenate(pattern_audio)
    
    def apply_fade(self, audio, fade_in=0.1, fade_out=0.1):
        """フェードイン・フェードアウトを適用"""
        length = len(audio)
        fade_in_samples = int(fade_in * self.sample_rate)
        fade_out_samples = int(fade_out * self.sample_rate)
        
        result = audio.copy()
        
        # フェードイン
        if fade_in_samples > 0:
            fade_curve = np.linspace(0, 1, fade_in_samples)
            result[:fade_in_samples] *= fade_curve
        
        # フェードアウト
        if fade_out_samples > 0:
            fade_curve = np.linspace(1, 0, fade_out_samples)
            result[-fade_out_samples:] *= fade_curve
        
        return result
    
    def mix_tracks(self, track_list, volumes=None):
        """複数のトラックをミキシング"""
        if volumes is None:
            volumes = [1.0] * len(track_list)
        
        # 最長のトラック長を取得
        max_length = max(len(track) for track in track_list)
        
        # 全トラックを同じ長さに調整してミックス
        mixed = np.zeros(max_length)
        for track, volume in zip(track_list, volumes):
            # トラックを最大長まで拡張（必要に応じて無音で埋める）
            extended_track = np.zeros(max_length)
            extended_track[:len(track)] = track
            mixed += extended_track * volume
        
        return mixed
    
    def normalize_audio(self, audio, target_peak=0.9):
        """音声を正規化"""
        peak = np.max(np.abs(audio))
        if peak > 0:
            return audio * (target_peak / peak)
        return audio

# ヘルパークラスのインスタンス作成
helpers = ProjectHelpers(config)

print("🎼 プロジェクトヘルパーが準備できました！")
print("利用可能な機能：")
print("- 和音生成 (create_chord)")
print("- メロディーシーケンス (create_melody_sequence)")
print("- ドラムパターン (create_drum_pattern)")
print("- フェード効果 (apply_fade)")
print("- トラックミキシング (mix_tracks)")
print("- 音声正規化 (normalize_audio)")

🎼 プロジェクトヘルパーが準備できました！
利用可能な機能：
- 和音生成 (create_chord)
- メロディーシーケンス (create_melody_sequence)
- ドラムパターン (create_drum_pattern)
- フェード効果 (apply_fade)
- トラックミキシング (mix_tracks)
- 音声正規化 (normalize_audio)


---

## 📋 最終プロジェクト：オリジナル楽曲制作

### プロジェクトの概要
これまで学んできた技術を使って、30秒〜1分程度のオリジナル楽曲を制作します。

### 制作のガイドライン
1. **楽曲構成**：イントロ、メインパート、アウトロの3部構成
2. **使用技術**：最低3つの技術要素を組み合わせる
   - 異なる波形（正弦波、のこぎり波、矩形波）
   - ADSR エンベロープ
   - フィルター効果
   - リバーブなどの空間系エフェクト
3. **創造性**：技術的な正確性だけでなく、音楽的な表現を重視
4. **プレゼンテーション**：作品の技術的説明と創作意図を発表

### 制作のステップ
1. **コンセプト決定**：どんな雰囲気の楽曲にするか
2. **メロディ作成**：主旋律の制作
3. **ハーモニー追加**：和音やベースラインの追加
4. **エフェクト適用**：音響効果の適用
5. **最終調整**：ミックスとバランス調整

---

## 🎨 制作のヒント・アイディア集

### 楽曲スタイルの例
- **アンビエント**：ゆったりとした浮遊感のある音楽
- **ミニマル**：シンプルなパターンの繰り返し
- **エレクトロニック**：リズミカルでエネルギッシュ
- **サウンドスケープ**：自然音や環境音をイメージ
- **8bit風**：レトロゲーム音楽風

In [10]:
# インスピレーション：簡単なアイディアサンプル

def create_inspiration_samples():
    """制作のヒントとなるサンプルを生成"""
    
    # 1. アンビエント風のコード進行
    print("🌙 アンビエント風サンプル")
    
    # C-E-Gの長三和音を生成
    chord1 = helpers.create_chord(note_to_frequency(note_name_to_number('C4')), 
                                  chord_type="major", duration=2.0, amplitude=0.4)
    
    # ADSR適用
    envelope = ADSREnvelope(attack=0.3, decay=0.2, sustain=0.5, release=0.5, config=config)
    env_data = envelope.generate(2.0)
    chord1 = apply_envelope(chord1, env_data)
    
    # フェード効果適用
    chord1 = helpers.apply_fade(chord1, fade_in=0.3, fade_out=0.5)
    
    display(Audio(chord1, rate=config.sample_rate))
    
    # 2. 8bit風のメロディ
    print("\n🎮 8bit風サンプル")
    melody_notes = ['C5', 'E5', 'G5', 'E5', 'F5', 'D5', 'C5']
    durations = [0.2] * len(melody_notes)
    
    # 矩形波風の音を作成
    melody = np.array([])
    envelope = ADSREnvelope(attack=0.01, decay=0.05, sustain=0.7, release=0.1, config=config)
    
    for note in melody_notes:
        freq = note_to_frequency(note_name_to_number(note))
        tone = square_osc.generate(freq, 0.2)
        tone = tone * 0.4  # 振幅調整
        env_data = envelope.generate(0.2)
        tone = apply_envelope(tone, env_data)
        melody = np.concatenate([melody, tone])
    
    display(Audio(melody, rate=config.sample_rate))
    
    # 3. ベースラインのパターン
    print("\n🎸 ベースライン サンプル")
    bass_notes = ['C2', 'C2', 'G2', 'F2']
    
    # のこぎり波でベースを生成
    bassline_audio = []
    envelope = ADSREnvelope(attack=0.01, decay=0.1, sustain=0.6, release=0.2, config=config)
    
    for note in bass_notes:
        freq = note_to_frequency(note_name_to_number(note))
        bass = saw_osc.generate(freq, 0.5)
        bass = bass * 0.6  # 振幅調整
        
        # ローパスフィルター適用
        lpf = LowPassFilter(cutoff_freq=300, config=config)
        bass = lpf.process(bass)
        
        # ADSR適用
        env_data = envelope.generate(0.5)
        bass = apply_envelope(bass, env_data)
        bassline_audio.append(bass)
    
    bassline = np.concatenate(bassline_audio)
    display(Audio(bassline, rate=config.sample_rate))

create_inspiration_samples()

🌙 アンビエント風サンプル



🎮 8bit風サンプル



🎸 ベースライン サンプル


---

## 🛠️ 楽曲制作支援ツール

In [6]:
# 楽曲制作支援ツール
import random

class CompositionHelper:
    """楽曲制作を支援するヘルパークラス"""
    
    def __init__(self, config):
        self.config = config
        
        # よく使用されるコード進行
        self.chord_progressions = {
            'pop': [['C4', 'E4', 'G4'], ['A3', 'C4', 'E4'], ['F3', 'A3', 'C4'], ['G3', 'B3', 'D4']],
            'sad': [['A3', 'C4', 'E4'], ['F3', 'A3', 'C4'], ['C4', 'E4', 'G4'], ['G3', 'B3', 'D4']],
            'happy': [['C4', 'E4', 'G4'], ['G3', 'B3', 'D4'], ['A3', 'C4', 'E4'], ['F3', 'A3', 'C4']],
            'mysterious': [['D4', 'F4', 'A4'], ['B3', 'D4', 'F4'], ['G3', 'B3', 'D4'], ['E3', 'G3', 'B3']]
        }
        
        # スケール（音階）
        self.scales = {
            'major': [0, 2, 4, 5, 7, 9, 11],
            'minor': [0, 2, 3, 5, 7, 8, 10],
            'pentatonic': [0, 2, 4, 7, 9],
            'blues': [0, 3, 5, 6, 7, 10]
        }
    
    def generate_chord(self, chord_notes, duration=1.0, waveform='sine', amplitude=0.3):
        """コードを生成"""
        chord = np.zeros(int(self.config.sample_rate * duration))
        
        for note in chord_notes:
            freq = note_to_frequency(note_name_to_number(note))
            
            if waveform == 'sine':
                tone = sine_osc.generate(freq, duration)
            elif waveform == 'sawtooth':
                tone = saw_osc.generate(freq, duration)
            elif waveform == 'square':
                tone = square_osc.generate(freq, duration)
            else:
                tone = sine_osc.generate(freq, duration)
            
            tone = tone * amplitude  # 振幅調整
            chord += tone
        
        return chord
    
    def generate_melody(self, scale='major', key='C', octave=4, length=8, note_duration=0.5):
        """ランダムなメロディを生成"""
        base_note = f'{key}{octave}'
        base_freq = note_to_frequency(note_name_to_number(base_note))
        scale_notes = self.scales[scale]
        
        melody = np.array([])
        envelope = ADSREnvelope(attack=0.05, decay=0.1, sustain=0.7, release=0.15, config=self.config)
        
        for _ in range(length):
            # スケール内からランダムに音を選択
            semitone_offset = random.choice(scale_notes)
            freq = base_freq * (2 ** (semitone_offset / 12))
            
            # 音符の生成
            note = sine_osc.generate(freq, note_duration)
            note = note * 0.4  # 振幅調整
            env_data = envelope.generate(note_duration)
            note = apply_envelope(note, env_data)
            
            melody = np.concatenate([melody, note])
        
        return melody
    
    def create_rhythm_pattern(self, bpm=120, measures=4, subdivisions=4):
        """リズムパターンを作成"""
        beat_duration = 60.0 / bpm  # 1拍の長さ（秒）
        subdivision_duration = beat_duration / subdivisions
        
        pattern = []
        for measure in range(measures):
            for beat in range(4):  # 4/4拍子
                for sub in range(subdivisions):
                    # ランダムにアクセントを配置
                    if beat == 0 and sub == 0:  # 強拍
                        pattern.append(1.0)
                    elif beat == 2 and sub == 0:  # 中強拍
                        pattern.append(0.7)
                    elif random.random() < 0.3:  # ランダムアクセント
                        pattern.append(0.5)
                    else:
                        pattern.append(0.0)  # 休符
        
        return pattern, subdivision_duration

# ヘルパーのインスタンス作成
composer = CompositionHelper(config)

print("🎼 楽曲制作支援ツールが準備できました！")
print("利用可能な機能：")
print("- コード生成")
print("- メロディ生成")
print("- リズムパターン生成")
print("- 様々な音階とコード進行")

🎼 楽曲制作支援ツールが準備できました！
利用可能な機能：
- コード生成
- メロディ生成
- リズムパターン生成
- 様々な音階とコード進行


In [7]:
# 楽曲制作ツールのクイックテスト
print("🧪 楽曲制作ツールのテスト")

# ADSRエンベロープのインスタンス作成
envelope = ADSREnvelope(attack=0.1, decay=0.2, sustain=0.6, release=0.3, config=config)

# 1. ポップスのコード進行
print("\n🎵 ポップスのコード進行")
progression = np.array([])
for chord_notes in composer.chord_progressions['pop']:
    chord = composer.generate_chord(chord_notes, duration=1.0, waveform='sine')
    env_data = envelope.generate(1.0)
    chord = apply_envelope(chord, env_data)
    progression = np.concatenate([progression, chord])

display(Audio(progression, rate=config.sample_rate))

# 2. ランダムメロディ
print("\n🎶 Cメジャースケールのランダムメロディ")
melody = composer.generate_melody(scale='major', key='C', octave=5, length=8)
display(Audio(melody, rate=config.sample_rate))

# 3. メロディ + コード
print("\n🎵 メロディ + コード（簡単なアレンジ）")
# 長さを合わせる
melody_short = composer.generate_melody(scale='major', key='C', octave=5, length=4, note_duration=1.0)
chord_short = np.array([])
for chord_notes in composer.chord_progressions['pop']:
    chord = composer.generate_chord(chord_notes, duration=1.0, waveform='sine', amplitude=0.2)
    env_data = envelope.generate(1.0)
    chord = apply_envelope(chord, env_data)
    chord_short = np.concatenate([chord_short, chord])

# メロディとコードをミックス
combined = helpers.mix_tracks([melody_short, chord_short], volumes=[0.8, 0.5])
combined = helpers.normalize_audio(combined, target_peak=0.8)

display(Audio(combined, rate=config.sample_rate))

🧪 楽曲制作ツールのテスト

🎵 ポップスのコード進行



🎶 Cメジャースケールのランダムメロディ



🎵 メロディ + コード（簡単なアレンジ）


---

## 🎯 あなたの作品制作エリア

### ステップ1：コンセプトの決定
まず、どんな楽曲を作りたいか決めましょう。以下のセルにコンセプトを記述してください。

**【あなたの楽曲コンセプト】**

楽曲のタイトル：

ジャンル・スタイル：

雰囲気・テーマ：

使用したい技術要素：
- [ ] 正弦波
- [ ] のこぎり波
- [ ] 矩形波
- [ ] ADSR エンベロープ
- [ ] フィルター
- [ ] リバーブ
- [ ] その他：

楽曲構成（予定）：
- イントロ（〜秒）：
- メインパート（〜秒）：
- アウトロ（〜秒）：

### ステップ2：メロディ・ハーモニーの制作

In [ ]:
# あなたの楽曲制作エリア - メロディ作成
print("🎵 メロディ制作エリア")

def create_my_melody():
    """あなたのオリジナルメロディを作成する関数"""
    
    # ここに好きな音階と音程を入力してください
    notes = ['C4', 'D4', 'E4', 'F4', 'G4', 'F4', 'E4', 'C4']
    durations = [0.5, 0.5, 0.5, 0.5, 1.0, 0.5, 0.5, 1.0]
    
    melody = np.array([])
    envelope = ADSREnvelope(attack=0.05, decay=0.1, sustain=0.7, release=0.2, config=config)
    
    for note, duration in zip(notes, durations):
        # 音程を周波数に変換
        freq = note_to_frequency(note_name_to_number(note))
        
        # 波形を選択（sine_osc, saw_osc, square_osc）
        tone = sine_osc.generate(freq, duration)
        tone = tone * 0.4  # 振幅調整
        
        # エンベロープを適用
        env_data = envelope.generate(duration)
        tone = apply_envelope(tone, env_data)
        
        melody = np.concatenate([melody, tone])
    
    return melody

# メロディの生成とテスト
my_melody = create_my_melody()
print("あなたのメロディ：")
display(Audio(my_melody, rate=config.sample_rate))

# メロディの波形を可視化
plt.figure(figsize=(12, 4))
time_axis = np.linspace(0, len(my_melody)/config.sample_rate, len(my_melody))
plt.plot(time_axis[:4410], my_melody[:4410])  # 最初の0.1秒を表示
plt.title('あなたのメロディの波形')
plt.xlabel('時間 (秒)')
plt.ylabel('振幅')
plt.grid(True)
plt.show()

In [ ]:
# あなたの楽曲制作エリア - ハーモニー（和音・ベース）作成
print("🎹 ハーモニー制作エリア")

def create_my_harmony():
    """あなたのオリジナルハーモニーを作成する関数"""
    
    # コード進行を選択または自作
    # composer.chord_progressions から選ぶか、独自に作成
    my_chords = [
        ['C3', 'E3', 'G3'],  # Cメジャー
        ['A2', 'C3', 'E3'],  # Aマイナー
        ['F2', 'A2', 'C3'],  # Fメジャー
        ['G2', 'B2', 'D3']   # Gメジャー
    ]
    
    harmony = np.array([])
    envelope = ADSREnvelope(attack=0.1, decay=0.2, sustain=0.6, release=0.3, config=config)
    
    for chord_notes in my_chords:
        # コードの生成
        chord = composer.generate_chord(chord_notes, duration=1.0, waveform='sine', amplitude=0.25)
        
        # エンベロープを適用
        env_data = envelope.generate(1.0)
        chord = apply_envelope(chord, env_data)
        
        # フィルターやエフェクトを適用（お好みで）
        # lpf = LowPassFilter(cutoff_freq=3000, config=config)
        # chord = lpf.process(chord)
        
        harmony = np.concatenate([harmony, chord])
    
    return harmony

def create_my_bass():
    """ベースラインを作成する関数"""
    
    bass_notes = ['C2', 'A1', 'F1', 'G1']  # ベース音程
    bassline = np.array([])
    envelope = ADSREnvelope(attack=0.01, decay=0.1, sustain=0.8, release=0.2, config=config)
    
    for note in bass_notes:
        freq = note_to_frequency(note_name_to_number(note))
        
        # ベースにはのこぎり波がよく合います
        bass_tone = saw_osc.generate(freq, 1.0)
        bass_tone = bass_tone * 0.4  # 振幅調整
        
        # ローパスフィルターで低域を強調
        lpf = LowPassFilter(cutoff_freq=300, config=config)
        bass_tone = lpf.process(bass_tone)
        
        # ベース用のエンベロープ
        env_data = envelope.generate(1.0)
        bass_tone = apply_envelope(bass_tone, env_data)
        
        bassline = np.concatenate([bassline, bass_tone])
    
    return bassline

# ハーモニーとベースの生成
my_harmony = create_my_harmony()
my_bass = create_my_bass()

print("あなたのハーモニー（コード）：")
display(Audio(my_harmony, rate=config.sample_rate))

print("\nあなたのベースライン：")
display(Audio(my_bass, rate=config.sample_rate))

### ステップ3：エフェクトの適用と最終ミックス

In [ ]:
# 最終ミックス - 全ての要素を組み合わせる
print("🎚️ 最終ミックス制作エリア")

def create_final_composition():
    """最終的な楽曲を制作する関数"""
    
    # 各トラックを再生成（または既存のものを使用）
    melody = create_my_melody()
    harmony = create_my_harmony()
    bass = create_my_bass()
    
    # ドラムパターンを追加（オプション）
    drum_pattern = ['K', '-', 'S', '-', 'K', '-', 'S', '-']  # キック-休符-スネア-休符のパターン
    drums = helpers.create_drum_pattern(drum_pattern, bpm=120, bars=2)
    
    # 全トラックの長さを揃える
    tracks = [melody, harmony, bass, drums]
    max_length = max(len(track) for track in tracks)
    
    # 各トラックを最大長に合わせる
    normalized_tracks = []
    for track in tracks:
        if len(track) < max_length:
            # 足りない部分を無音で埋める
            padded = np.zeros(max_length)
            padded[:len(track)] = track
            normalized_tracks.append(padded)
        else:
            normalized_tracks.append(track[:max_length])
    
    # ボリューム調整
    volumes = [0.7, 0.4, 0.5, 0.6]  # メロディ, ハーモニー, ベース, ドラム
    
    # トラックをミックス
    final_mix = helpers.mix_tracks(normalized_tracks, volumes)
    
    # フェードイン・フェードアウト適用
    final_mix = helpers.apply_fade(final_mix, fade_in=0.5, fade_out=1.0)
    
    # 最終的な正規化
    final_mix = helpers.normalize_audio(final_mix, target_peak=0.8)
    
    return final_mix

# 最終楽曲を制作
final_composition = create_final_composition()

print("\n🎉 完成！あなたのオリジナル楽曲：")
display(Audio(final_composition, rate=config.sample_rate))

# 楽曲の長さを表示
duration = len(final_composition) / config.sample_rate
print(f"\n📊 楽曲情報：")
print(f"  - 長さ: {duration:.2f}秒")
print(f"  - サンプル数: {len(final_composition):,}")
print(f"  - 最大振幅: {np.max(np.abs(final_composition)):.3f}")

In [ ]:
# 最終楽曲の可視化と分析
print("📈 楽曲の可視化と分析")

# 1. 時間軸での波形表示
plt.figure(figsize=(15, 10))

# 全体の波形
plt.subplot(3, 1, 1)
time_axis = np.linspace(0, len(final_composition)/config.sample_rate, len(final_composition))
plt.plot(time_axis, final_composition, color='blue', alpha=0.7)
plt.title('あなたの楽曲 - 全体波形')
plt.xlabel('時間 (秒)')
plt.ylabel('振幅')
plt.grid(True, alpha=0.3)

# 周波数スペクトラム
plt.subplot(3, 1, 2)
frequencies = np.fft.fftfreq(len(final_composition), 1/config.sample_rate)
spectrum = np.abs(np.fft.fft(final_composition))
plt.plot(frequencies[:len(frequencies)//2], spectrum[:len(spectrum)//2])
plt.title('周波数スペクトラム')
plt.xlabel('周波数 (Hz)')
plt.ylabel('強度')
plt.xlim(0, 2000)  # 低域〜中域を表示
plt.grid(True, alpha=0.3)

# スペクトログラム（時間-周波数分析）
plt.subplot(3, 1, 3)
f, t, Sxx = signal.spectrogram(final_composition, config.sample_rate, nperseg=1024)
plt.pcolormesh(t, f, 10 * np.log10(Sxx), shading='gouraud')
plt.title('スペクトログラム（時間-周波数分析）')
plt.xlabel('時間 (秒)')
plt.ylabel('周波数 (Hz)')
plt.ylim(0, 2000)
plt.colorbar(label='強度 (dB)')

plt.tight_layout()
plt.show()

# 楽曲の音響特性分析
print("\n🔍 音響特性分析：")
print(f"  - RMS値: {np.sqrt(np.mean(final_composition**2)):.4f}")
print(f"  - ピーク値: {np.max(np.abs(final_composition)):.4f}")
print(f"  - ダイナミックレンジ: {20*np.log10(np.max(np.abs(final_composition))/np.sqrt(np.mean(final_composition**2))):.2f} dB")

# 楽曲をファイルとして保存（オプション）
try:
    save_audio(final_composition, "my_composition.wav", config.sample_rate)
    print(f"\n💾 楽曲が 'my_composition.wav' として保存されました！")
except Exception as e:
    print(f"\n⚠️ 保存エラー: {e}")
    print("  ローカル環境では保存機能が利用できます。")

---

## 🎤 発表準備エリア

### あなたの作品について
作品を発表する準備をしましょう。以下の項目を埋めてください：

**【作品発表シート】**

**作品タイトル：**


**制作コンセプト・テーマ：**


**使用した技術要素：**
- 波形の種類：
- エンベロープの設定：
- フィルター：
- エフェクト：
- その他：

**技術的な工夫・特徴：**


**制作で苦労した点：**


**音楽的な表現で重視した点：**


**今後改良したい点：**


**感想・学んだこと：**

---

## 🌟 ボーナスチャレンジ

時間に余裕がある方は、以下のチャレンジに挑戦してみてください：

### チャレンジ1：アレンジバージョン
同じメロディで異なるアレンジ（テンポ、楽器、エフェクト）を作成

In [ ]:
# チャレンジ1：アレンジバージョン制作エリア
print("🎵 アレンジバージョン制作")

def create_arrangement_version():
    """オリジナル楽曲のアレンジバージョンを作成"""
    
    # 例：テンポを変える、波形を変える、エフェクトを強くする等
    
    # アレンジのアイディア：
    # 1. 全て矩形波にして8bit風に
    # 2. テンポを半分にしてアンビエント風に
    # 3. リバーブを強くして宇宙的な感じに
    
    # ここにアレンジコードを書いてください
    pass

# アレンジバージョンの制作（自由にコーディング）

### チャレンジ2：インタラクティブ要素
ランダム要素やパラメータを変更できる楽曲を作成

In [ ]:
# チャレンジ2：インタラクティブ楽曲制作エリア
print("🎲 インタラクティブ楽曲制作")

def create_interactive_music(randomness=0.5, filter_freq=1000, reverb_mix=0.3):
    """パラメータによって変化する楽曲を作成"""
    
    # パラメータに応じて楽曲が変化するシステム
    # randomness: ランダム要素の強さ (0-1)
    # filter_freq: フィルター周波数
    # reverb_mix: リバーブの強さ
    
    # ここにインタラクティブ楽曲のコードを書いてください
    pass

# 異なるパラメータで楽曲を生成
print("パラメータセット1:")
# create_interactive_music(randomness=0.2, filter_freq=2000, reverb_mix=0.1)

print("\nパラメータセット2:")
# create_interactive_music(randomness=0.8, filter_freq=500, reverb_mix=0.6)

---

## 🤔 振り返りと今後の学習

### 7回の講座を通じて学んだこと

**技術的スキル：**
- 基本波形の生成と特性理解
- エンベロープによる音の動的制御
- フィルターによる音色変化
- 音響効果の実装と応用
- MIDIとシーケンシング
- サンプリングとリアルタイム処理
- 楽曲制作とミキシング

**音楽的理解：**
- 音の物理的性質と知覚の関係
- エレクトロニック音楽の制作手法
- 音響効果の音楽的応用
- 作曲とアレンジメントの基礎

**創造的思考：**
- 技術と創造性の融合
- 制約の中での表現力
- 試行錯誤による学習
- 作品制作プロセスの体験

### 今後の学習方向

**より高度な音響プログラミング：**
- 高度なシンセサイザー設計
- リアルタイム音声処理
- 機械学習を使った音楽生成
- VSTプラグイン開発

**音楽理論の深化：**
- 和声学・対位法
- 音楽分析と楽式
- 様々な音楽ジャンルの研究
- 音響心理学

**実践的なスキル：**
- DAW（Digital Audio Workstation）の習得
- レコーディング技術
- ミキシング・マスタリング
- ライブパフォーマンス

### 推奨リソース

**書籍：**
- "Computer Music: Synthesis, Composition, and Performance" - Dodge & Jerse
- "The Audio Programming Book" - Boulanger & Lazzarini
- "音響学入門" - 日本音響学会

**ソフトウェア：**
- SuperCollider（音響プログラミング）
- Pure Data（ビジュアル音響プログラミング）
- Max/MSP（リアルタイム音響処理）
- Reaper, Ableton Live（DAW）

**オンラインリソース：**
- Coursera: Audio Signal Processing for Music Applications
- YouTube: Making Music with Python
- GitHub: audio-programming repositories

---

## 🎊 コース修了おめでとうございます！

### 最終メッセージ

7回にわたる音響プログラミング基礎セミナーを通じて、皆さんは単なる技術習得以上のものを手に入れました：

**🔧 技術的基盤：** 音響プログラミングの基本的な概念と実装手法

**🎵 音楽的感性：** 技術と芸術性を融合させる視点

**🧠 創造的思考：** 制約の中で新しい表現を見つける能力

**🤝 協働経験：** 作品を共有し、お互いから学ぶ姿勢

### これからの音楽制作の旅

今日制作した作品は、皆さんの音楽制作の旅の始まりです。プログラミングによる音楽制作は無限の可能性を秘めています。従来の楽器や音楽制作ツールでは実現できない、まったく新しい音楽表現を創造することができます。

技術的な知識を深めることも大切ですが、それ以上に大切なのは**実際に音を作り続けること**です。日々の小さな実験から、思わぬ発見や創造が生まれます。

### 最後に

音楽とプログラミングの融合は、単なる技術的な組み合わせではありません。それは**人間の創造性を拡張し、新しい芸術的可能性を開拓する冒険**です。

皆さんが今後も音響プログラミングを通じて、自分だけの音楽的表現を発見し、発展させていくことを心から願っています。

**🎵 Happy Coding & Happy Music Making! 🎵**

---

*この講座が皆さんの音楽制作の新しい扉を開くきっかけとなれば幸いです。*